# APIM ❤️ OpenAI

## Backend pool Load Balancing lab
![flow](images/backend-pool-load-balancing.gif)

Playground to try the built-in load balancing [backend pool functionality of APIM](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep) to either a list of Azure OpenAI endpoints or mock servers.

Notes:
- The backend pool uses round-robin by default
- But priority and weight based routing are also supported: Adjust the `priority` (the lower the number, the higher the priority) and `weight` parameters in the `openai_resources` variable
- The `retry` API Management policy initiates a retry to an available backend if an HTTP 429 status code is encountered

<a id='2'></a>
### 2️⃣ Create deployment using Terraform

This lab uses Terraform to declaratively define all the resources that will be deployed. Change the [variables.tf](variables.tf) directly to try different configurations.

In [ ]:
! terraform init
! terraform apply -auto-approve

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [14]:
apim_resource_gateway_url = ! terraform output -raw apim_resource_gateway_url
apim_resource_gateway_url = apim_resource_gateway_url.n
print("👉🏻 APIM Resource Gateway URL: ", apim_resource_gateway_url)

app_insights_app_id = ! terraform output -raw app_insights_app_id
app_insights_app_id = app_insights_app_id.n
print("👉🏻 Application Insights App ID: ", app_insights_app_id)

apim_subscription_key_1 = ! terraform output -raw apim_subscription_key_1
apim_subscription_key_1 = apim_subscription_key_1.n
print("👉🏻 APIM Subscription Key 1: ", apim_subscription_key_1)

apim_subscription_key_2 = ! terraform output -raw apim_subscription_key_2
apim_subscription_key_2 = apim_subscription_key_2.n
print("👉🏻 APIM Subscription Key 2: ", apim_subscription_key_2)

apim_subscription_key_3 = ! terraform output -raw apim_subscription_key_3
apim_subscription_key_3 = apim_subscription_key_3.n
print("👉🏻 APIM Subscription Key 3: ", apim_subscription_key_3)

resource_group_name = ! terraform output -raw resource_group_name
resource_group_name = resource_group_name.n
print("👉🏻 Resource Group Name: ", resource_group_name)

app_insights_resource_name = ! terraform output -raw app_insights_resource_name
app_insights_resource_name = app_insights_resource_name.n
print("👉🏻 Application Insights Resource Name: ", app_insights_resource_name)

openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"
openai_deployment_name = "gpt-4o"

👉🏻 APIM Resource Gateway URL:  https://apim-genai-320-03.azure-api.net
👉🏻 Application Insights App ID:  09361747-7755-43ae-af86-c2327d97eef2
👉🏻 APIM Subscription Key 1:  8b755e4d29bd4270a25f89c785d671db
👉🏻 APIM Subscription Key 2:  e1a68a747743473f98e2593fb8692ca1
👉🏻 APIM Subscription Key 3:  b91719ba1575486e9a74f0d177d905f3
👉🏻 Resource Group Name:  rg-apim-genai-320-03
👉🏻 Application Insights Resource Name:  app-insights


## ⚠️ Important note about API Management Subscriptions

You will need to go to the API Management instance in the Azure portal and click each one of the three subscriptions and save them to activate them. There might be a bug preventing subscriptions to be validated.

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [15]:
import time
import os
import json
import datetime
import requests

runs = 10
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
api_runs = []

for i in range(runs):
    print("▶️ Run:", i+1, "/", runs)
    

    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    start_time = time.time()
    response = requests.post(url, headers = {'api-key':apim_subscription_key_1}, json = messages)
    response_time = time.time() - start_time
    
    print(f"⌚ {response_time:.2f} seconds")
    # Check the response status code and apply formatting
    if 200 <= response.status_code < 300:
        status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
    elif response.status_code >= 400:
        status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
    else:
        status_code_str = str(response.status_code)  # No formatting

    # Print the response status with the appropriate formatting
    print("Response status:", status_code_str)
    
    print("Response headers:", response.headers)
    
    if "x-ms-region" in response.headers:
        print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # this header is useful to determine the region of the backend that served the request
        api_runs.append((response_time, response.headers.get("x-ms-region")))
    
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("Token usage:", data.get("usage"), "\n")
        print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
    else:
        print(response.text)   

▶️ Run: 1 / 10
⌚ 6.16 seconds
Response status: 200 - OK
Response headers: {'Content-Type': 'application/json', 'Date': 'Fri, 20 Dec 2024 09:07:12 GMT', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': 'a2b814c4-a580-4845-9b70-5af14f7381a9', 'x-ratelimit-remaining-requests': '79', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '34a6dc17-477a-4043-963a-7c771cd6d5ad', 'X-Content-Type-Options': 'nosniff', 'azureml-model-session': 'd055-20241120015010', 'x-ms-region': 'UK South', 'x-envoy-upstream-service-time': '1311', 'x-ms-client-request-id': 'Not-Set', 'x-ratelimit-remaining-tokens': '7340', 'Request-Context': 'appId=cid-v1:09361747-7755-43ae-af86-c2327d97eef2'}
x-ms-region: UK South
Token usage: {'completion_tokens': 41, 'prompt_tokens': 30, 'total_tokens': 71} 

💬  Oh, of course! Because clearly, I’m d

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [16]:
import time
from openai import AzureOpenAI

runs = 3

for i in range(runs):
    print("▶️ Run: ", i+1)

    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]

    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key_3,
        api_version=openai_api_version
    )

    start_time = time.time()

    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    print("💬 ", response.choices[0].message.content)


▶️ Run:  1
⌚ 2.06 seconds
💬  Oh, of course! Let me just ask my crystal ball... Oh wait, I'm just an AI text model without any concept of time. But I’m sure you’re surrounded by gadgets that can tell you the time. Isn't technology wonderful?
▶️ Run:  2
⌚ 6.03 seconds
💬  Oh, absolutely! Let me just teleport to your location and check the nearest clock. Or, you know, you could just glance at the device you’re using to talk to me. That might be quicker.
▶️ Run:  3
⌚ 2.71 seconds
💬  Oh, absolutely! Let me just check my imaginary clock... Oh wait, I'm an AI without access to real-time data. You'll have to rely on something a bit more tangible, like your phone or a clock. Good luck!


<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [17]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_insights_app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


,timestamp,apiName,apimSubscription,duration,userAgent,model,messages,completion,region,promptTokens,completionTokens,totalTokens,remainingTokens,remainingRequests
0,2024-12-20T09:07:26.9910117Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,897.20,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure, let me just check my invisible watch...",Sweden Central,30,41,71,7340,79
1,2024-12-20T09:07:25.1806668Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1628.92,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just dust off my crystal ball ...",France Central,30,38,68,7340,79
2,2024-12-20T09:07:23.3467327Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1633.61,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure! Let me just glance at my imaginary w...",UK South,30,39,69,2720,72
3,2024-12-20T09:07:21.5012722Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1654.49,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, because I'm just sitting here with my...",UK South,30,44,74,3380,73
4,2024-12-20T09:07:19.7864578Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1530.19,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure! Just let me whip out my magic crysta...",UK South,30,42,72,4040,74
5,2024-12-20T09:07:17.815167Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1732.19,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Of course, let me just check my invisible watc...",UK South,30,45,75,4700,75
6,2024-12-20T09:07:16.0188953Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1594.90,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Because I'm totally a clock. Ju...",UK South,30,35,65,5360,76
7,2024-12-20T09:07:14.1017384Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,1714.88,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Because I totally have a magica...",UK South,30,32,62,6020,77
8,2024-12-20T09:07:13.0332883Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,867.37,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, let me just use my magical powers to ...",UK South,30,46,76,6680,78
9,2024-12-20T09:07:08.7240437Z,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,4071.57,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, of course! Because clearly, I�m directly c...",UK South,30,41,71,7340,79


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [18]:
import time
from openai import AzureOpenAI
runs = 2

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_1, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 1: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_2, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 2: ", response.choices[0].message.content)

    client = AzureOpenAI(azure_endpoint=apim_resource_gateway_url, api_key=apim_subscription_key_3, api_version=openai_api_version)
    response = client.chat.completions.create(model=openai_model_name, messages=messages, extra_headers={"x-user-id": "alex"})
    print("💬 ","for subscription 3: ", response.choices[0].message.content)


▶️ Run:  1
💬  for subscription 1:  Oh sure, let me just utilize my non-existent ability to look at clocks. Unfortunately for you, I'm just trapped in this screen with no idea what time reality is experiencing right now. Why not try something groundbreaking, like checking a clock or your phone?
💬  for subscription 2:  Oh sure, let me just consult my imaginary watch. Is that 8 o'clock in the land of make-believe? You might want to try a clock or your phone for something a bit more accurate.
💬  for subscription 3:  Oh, of course! Let me just check my non-existent watch. Actually, why don't you do something radical and look at your phone or a clock yourself? It's amazing, really.
▶️ Run:  2
💬  for subscription 1:  Oh, of course! Let me just glance at my invisible watch for you. Sorry, looks like I can't actually tell time. Maybe check a clock? They're quite good at that sort of thing!
💬  for subscription 2:  Oh sure, let me just reach through the internet and check that for you. Or, you kn

<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM.

In [19]:
import pandas as pd
import json

query = "\"" + "customMetrics  \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

result_stdout = ! az monitor app-insights query --app {app_insights_resource_name} -g {resource_group_name} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
df = pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


,timestamp,value,clientIP,apiId,apimSubscription,UserId
0,09:07,703,176.177.25.47,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,N/A
1,09:08,67,176.177.25.47,api-azure-openai,c6a0b2b6-6e60-47f9-8622-0aaf1759de86,alex
2,09:08,151,176.177.25.47,api-azure-openai,3e8b2a2c-04d4-4d6d-b188-17c063154ed3,alex
3,09:08,72,176.177.25.47,api-azure-openai,0328e2c9-cbcc-460e-8831-c387b509510a,alex
4,09:08,226,176.177.25.47,api-azure-openai,c6a0b2b6-6e60-47f9-8622-0aaf1759de86,N/A


## View logs and metrics on Azure Managed Grafana dashboard

You can also view the logs and metrics on the Azure Managed Grafana dashboard. Navigate to the Azure Managed Grafana. Login, then go to the dashboards section. Then import the following 2 dashboards:
1. Dashboard with ID : 16604 to vider logs and metrics from API Management.
2. Dashboard from JSON file [./grafana-dashboard-apim-openai.json](./grafana-dashboard-apim-openai.json) to view logs and metrics from Application Insights.
   Alternatively, you can also import the dashbord from the [Grafana dashboard ID: 22552](https://grafana.com/grafana/dashboards/22552).

![](images/grafana-dashboard-openai.png)